# Projeto 1 - Ciência dos Dados

Nome: Venâncio Freitas de Araújo Filho

Nome: WELLINGTON RODRIGUES DA SILVA

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

## 1. Importando as Bibliotecas e fazendo um reconhecimentos das bases de dados.

___
Carregando algumas bibliotecas:

In [65]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

import nltk
from nltk.stem import PorterStemmer
nltk.download('punkt')
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import bigrams
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\venan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\venan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\venan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [66]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\venan\OneDrive - Insper - Institudo de Ensino e Pesquisa\Materias 2° semestre Insper\Ciência de dados\Projetos\Classificador-Naive-Bayes-Amazon\Código_Venâncio


Dados de Treino: 

Carregando a base de dados com os tweets classificados manualmente:

In [67]:
train = pd.read_excel('dados_treino.xlsx')
train

,Mensagem,Target
0,Muito erros grosseiros de tradução. O material...,Editora
1,"Primeiramente meu pedido nem veio,Mas me reemb...",Amazon
2,"Livro com um monte de clichês, mas com pouco c...",Editora
3,Sim.adorei o livro Vou continuar a comprar o r...,Editora
4,Não recebi o livro . Não sei o motivo. Foi fei...,Amazon
...,...,...
293,O livro foi classificado como novo no site e c...,Amazon
294,"Não gostei porquê a capa veio com um rasgo, o ...",Amazon
295,EU NEM SE QUER RECEBI O PRODUTO! Voces são mui...,Amazon
296,Não consigo entender o alto valor desses e-boo...,Amazon


- Avaliando a Proporção dos valores

In [68]:
train.Target.value_counts(True).round(3)

Autor      0.377
Editora    0.311
Amazon     0.311
Name: Target, dtype: float64

Dados de Teste


In [69]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,Target
0,Amontoado de proposições absurdas destinadas a...,Autor
1,Quando comprei esse livro fui com uma grande e...,Autor
2,Li um terço do livro. Arrastei-me para procura...,Autor
3,"Sensacionalista, raso e cheio de distorções ma...",Autor
4,A mensagem do livro da pra entender em 1 págin...,Autor


- Analisando a proporção dos valores 

In [70]:
test.Target.value_counts()

Autor      102
Editora     76
Amazon      33
Name: Target, dtype: int64

In [71]:
test.Target

0      Autor
1      Autor
2      Autor
3      Autor
4      Autor
       ...  
206    Autor
207    Autor
208    Autor
209    Autor
210    Autor
Name: Target, Length: 211, dtype: object

___
## Desenvolvimento de Metodologia

Tendo em vista que a tarefa submetida tange o treinamento do bot para a classificação das mensagens de acordo com as palavras envolvidas, o grupo identificou como relevante treinar o robô para, a partir das palavras envolvidas na mensagem, identificar para quais grupos seria adequado encaminhar a mensagem. Assim, utilizou-se de 3 formas de classificação: 



- Amazon: Mensagens relacionadas ao serviço oferecido pela Plataforma; 
- Editora: Mensagens relacionadas aos serviços prestados pela editora; 
- Autor: Mensagens relacionadas aos serviços prestados pelo autor ao livro.


___
### Montando um Classificador Naive-Bayes

- Nesta etapa, vamos iniciar mostrando os rótulos e às frequências referentes à cada rótulo para o classificador, afim de treiná-lo.

## 1.Vamos Iniciar Extraindo os Dados Referente à cada rótulo


# 1.1 Funções de limpeza do texto: 

# Função CleanUp 1 : Retirada de pontuações

In [72]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$'']' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

## Função Clean UP 2: Stemming

In [73]:
def steamming(texto): #Recebe lista 
    stemmed_palavras = []
    stemmer = PorterStemmer()
    for palavra in texto:
        
        stemmed_palavras.append(stemmer.stem(palavra)) 
    
    return stemmed_palavras


## Função Clean UP 3: Stop Words

In [74]:
def stopwords2(texto): #Recebe lista 
    palavras_sem_stopwords = []
    palavras = word_tokenize(' '.join(texto), language='portuguese')

    stopwords_pt = set(stopwords.words('portuguese'))
    
    for palavra in palavras:
        if palavra not in stopwords_pt:
            palavras_sem_stopwords.append(palavra)
    
    return palavras_sem_stopwords

## Função Clean UP 4:  Remoção de Emojis

In [75]:
def remove_emojis(texto): #Recebe lista 
    texto = ' '.join(texto)
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # Emoticons
                           u"\U0001F300-\U0001F5FF"  # Símbolos e pictogramas diversos
                           u"\U0001F700-\U0001F77F"  # Símbolos alquímicos
                           u"\U0001F780-\U0001F7FF"  # Símbolos de palavras
                           u"\U0001F800-\U0001F8FF"  # Símbolos de árabes estendidos-A
                           u"\U0001F900-\U0001F9FF"  # Símbolos suplementares de árabes estendidos-B
                           u"\U0001FA00-\U0001FA6F"  # Símbolos de xadrez
                           u"\U0001FA70-\U0001FAFF"  # Símbolos suplementares de xadrez
                           u"\U0001F004-\U0001F0CF"  # Símbolos de domino
                           u"\U0001F170-\U0001F251"  # Símbolos de tai-xi
                           u"\U0001F300-\U0001F5FF"  # Símbolos e pictogramas diversos
                           u"\U0001F600-\U0001F64F"  # Emoticons
                           u"\U0001F680-\U0001F6FF"  # Transporte e mapas
                           u"\U0001F700-\U0001F77F"  # Símbolos alquímicos
                           u"\U0001F780-\U0001F7FF"  # Símbolos de palavras
                           u"\U0001F800-\U0001F8FF"  # Símbolos de árabes estendidos-A
                           u"\U0001FA00-\U0001FA6F"  # Símbolos de xadrez
                           u"\U0001FA70-\U0001FAFF"  # Símbolos suplementares de xadrez
                           "]+", flags=re.UNICODE)
    emoji_pattern = emoji_pattern.sub(r'', texto)
    emoji_pattern = emoji_pattern.split(' ')
    return emoji_pattern


## Função 5: Tokenização Por bigramas

In [76]:
def tokenize(texto):
    ' '.join(texto)
    lista_bigramas = list(bigrams(texto))
    return lista_bigramas

## Iniciando a extração dos textos referentes aos Rótulos

# Função de extração 1: Limpeza do Dataframe: 


In [77]:
def extracao_texto(series): 
    texto_novo = ''
    for linha in series:
        texto_novo += linha + ' '
    return texto_novo 

# Função de Padronização da Extração dos Rótulos: 


In [78]:
def transforma_rotulo(dataframe , string_rotulo):
    dados_rotulo = dataframe.loc[dataframe.Target == string_rotulo , : ]
    texto_rotulo = extracao_texto(dados_rotulo.Mensagem)
    texto_rotulo = cleanup(texto_rotulo)
    texto_rotulo = texto_rotulo.lower()
    texto_rotulo = texto_rotulo.split(' ')
    texto_rotulo = remove_emojis(texto_rotulo)
    #texto_rotulo = stopwords2(texto_rotulo)
    texto_rotulo = steamming(texto_rotulo)
    #texto_rotulo = tokenize(texto_rotulo)
    tabela_rotulo = pd.Series(texto_rotulo)
    return [tabela_rotulo , texto_rotulo]


# 1. Tratando dados do Autor 

- Extraindo os dados e implementando as limpezas. 

In [79]:
#Montando um Texto
tabela_autor = transforma_rotulo(train , 'Autor')[0]
texto_autor = transforma_rotulo(train , 'Autor')[1]
print(tabela_autor.shape)

(6683,)


- Tabela de palavras com frequência absoluta

In [80]:
#Frequências Absolutas 
tabela_autor_abs = tabela_autor.value_counts()
tabela_autor_abs

de             256
o              237
que            223
e              202
a              196
              ... 
invençõ          1
passaria         1
praticament      1
apontar          1
sentimento       1
Length: 1848, dtype: int64

- Tabela de palavras com Frequência relativa

In [81]:
#Frequências relativas 
tabela_autor_relativa = tabela_autor.value_counts(True)
tabela_autor_relativa 

de             0.038306
o              0.035463
que            0.033368
e              0.030226
a              0.029328
                 ...   
invençõ        0.000150
passaria       0.000150
praticament    0.000150
apontar        0.000150
sentimento     0.000150
Length: 1848, dtype: float64

- Número de palavras no Dataframe

In [82]:
tabela_autor_abs.sum()

6683

## 2. Dados da Editora 

- Extração dos dados da Editora

In [83]:
tabela_editora = transforma_rotulo(train , 'Editora')[0]
texto_editora = transforma_rotulo(train , 'Editora')[1]
print(tabela_editora.head())
print(tabela_editora.shape)

0        muito
1         erro
2    grosseiro
3           de
4     tradução
dtype: object
(4727,)


- Tabela de frequências absolutas das palavras 

In [84]:
#Frequência absoluta
tabela_editora_abs = tabela_editora.value_counts()
tabela_editora_abs

de           194
a            158
o            139
que          134
e            132
            ... 
avaliação      1
reduza         1
dezena         1
resisti        1
frustada       1
Length: 1520, dtype: int64

- Tabela de Frequências relativas das palavras

In [85]:
#Frequência Relativa 
tabela_editora_relativa = tabela_editora.value_counts(True)
tabela_editora_relativa

de           0.041041
a            0.033425
o            0.029406
que          0.028348
e            0.027925
               ...   
avaliação    0.000212
reduza       0.000212
dezena       0.000212
resisti      0.000212
frustada     0.000212
Length: 1520, dtype: float64

In [86]:
#Quantidade de palavras 
tabela_editora_abs.sum()

4727

# 3. Dados da Amazon


- Extraindo os dados 

In [87]:
tabela_amazon = transforma_rotulo(train , 'Amazon')[0]
texto_amazon = transforma_rotulo(train , 'Amazon')[1]
print(tabela_amazon.shape)

(4691,)


 - Frequências absolutas das palavras:  

In [88]:
#Frequência absoluta 
tabela_amazon_abs = tabela_amazon.value_counts()
tabela_amazon_abs

o              177
a              157
e              143
que            129
de             104
              ... 
cópia            1
enviada          1
lamppm           1
responsável      1
otimo            1
Length: 1280, dtype: int64

- Frequência relativa das palavras

In [89]:
#Frequência relativa
tabela_amazon_relativa = tabela_amazon.value_counts(True)
tabela_amazon_relativa

o              0.037732
a              0.033468
e              0.030484
que            0.027499
de             0.022170
                 ...   
cópia          0.000213
enviada        0.000213
lamppm         0.000213
responsável    0.000213
otimo          0.000213
Length: 1280, dtype: float64

- Número de palavras

In [90]:
tabela_amazon_abs.sum()

4691

___
# Construindo o Classificador a partir dos testes

Agora você deve testar o seu classificador com a base de Testes.

In [91]:
def lista_sem_repetição(lista_palavras_totais):
    lista_sem_repeticao = []
    for palavras in lista_palavras_totais: 
        if palavras not in lista_sem_repeticao:
            lista_sem_repeticao.append(palavras)
    return lista_sem_repeticao

In [92]:
todas_palavras = texto_editora + texto_amazon + texto_autor
lista_sem_repeticao = lista_sem_repetição(todas_palavras)
len(lista_sem_repeticao)

3347

# 1 Probabilidade de Cada Rótulo dentro do texto

In [93]:
def prob_rotulo(lista_todas_palavras , tabela_rotulos):
    probabilidade = sum(tabela_rotulos) / len(lista_todas_palavras)
    return probabilidade

In [94]:
P_Autor = prob_rotulo(todas_palavras , tabela_autor_abs)
P_Autor

0.41506738711881247

In [95]:
P_Amazon = prob_rotulo(todas_palavras , tabela_amazon_abs)
P_Amazon

0.291348363455686

In [96]:
P_Editora = prob_rotulo(todas_palavras , tabela_editora_abs)
P_Editora

0.29358424942550154

# 2. Probabilidade de aparecer em cada frase

# Função da Suavização 

In [97]:
def suavizacao(frase_linha , tabela_frequencia_abs , alpha , lista_sem_repetição , lista_palavras_rotulos):
    P_frase_dado_rótulo = 1 
    for palavra in frase_linha: 
        if palavra in lista_palavras_rotulos: 
            P_frase_dado_rótulo *= (tabela_frequencia_abs[palavra] + alpha)/ ( (len(lista_sem_repeticao) * alpha) + len(lista_palavras_rotulos))
        else: 
            P_frase_dado_rótulo *= alpha/(len(lista_palavras_rotulos) +(len(lista_sem_repeticao)* alpha))
    return P_frase_dado_rótulo

In [98]:
def vencedor (lista_probs):
    if max(lista_probs) == lista_probs[0]:
        return 'Autor'
    elif max(lista_probs) == lista_probs[1]:
        return 'Editora'
    elif max(lista_probs) == lista_probs[2]:
        return 'Amazon'
    

In [99]:
def loop_bot(dataframe , alpha):
    P_frase_dado_autor = 1
    P_frase_dado_editora = 1
    P_frase_dado_amazon = 1
    lista_teste = []
    lista_probs = []
    for frase in dataframe.Mensagem:
        #CleanUps
        frase = cleanup(str(frase))
        frase = frase.lower()
        frase = frase.split(' ')
        frase = remove_emojis(frase)
        frase = steamming(frase)
        
        #Probabilidade de frase dado rótulo
        P_frase_dado_autor = suavizacao(frase , tabela_autor_abs , 1 , lista_sem_repeticao , texto_autor)
        P_frase_dado_editora = suavizacao(frase , tabela_editora_abs , 1 , lista_sem_repeticao , texto_editora)    
        P_frase_dado_amazon = suavizacao(frase, tabela_amazon_abs , 1 , lista_sem_repeticao , texto_amazon) 
        
        #Probabilidade de rótulo dado frase         
        P_autor_dado_frase = P_frase_dado_autor * P_Autor
        lista_probs.append( P_autor_dado_frase)
    
        P_editora_dado_frase = P_frase_dado_editora * P_Editora
        lista_probs.append( P_editora_dado_frase)
    
        P_amazon_dado_frase =  P_frase_dado_amazon * P_Amazon
        lista_probs.append( P_amazon_dado_frase)
        
        #Definindo o resultado do Robô
        maior = vencedor(lista_probs)
        lista_teste.append(maior)
    
        #Reiniciando as variáveis 
        P_frase_dado_editora = 1
        P_frase_dado_amazon = 1
        P_frase_dado_autor = 1
    
        P_autor_dado_frase = 1
    
        P_editora_dado_frase = 1
    
        P_amazon_dado_frase =  1
        lista_probs = []
    return lista_teste 

In [100]:
lista_teste = loop_bot(test , 1)
lista_teste

['Editora',
 'Autor',
 'Autor',
 'Editora',
 'Autor',
 'Autor',
 'Autor',
 'Amazon',
 'Amazon',
 'Autor',
 'Editora',
 'Autor',
 'Amazon',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Editora',
 'Autor',
 'Autor',
 'Autor',
 'Amazon',
 'Autor',
 'Amazon',
 'Autor',
 'Autor',
 'Autor',
 'Editora',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Editora',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Amazon',
 'Autor',
 'Autor',
 'Autor',
 'Amazon',
 'Autor',
 'Autor',
 'Amazon',
 'Amazon',
 'Autor',
 'Autor',
 'Autor',
 'Autor',
 'Editora',
 'Autor',
 'Autor',
 'Amazon',
 'Autor',
 'Amazon',
 'Amazon',
 'Editora',
 'Autor',
 'Amazon',
 'Autor',
 'Amazon',
 'Autor',
 'Amazon',
 'Autor',
 'Amazon',
 'Amazon',
 'Amazon',
 'Autor',
 'Amazon',
 'Amazon',
 'Editora',
 'Autor',
 'Autor',
 'Autor',
 'Amazon',
 'Autor',
 'Amazon',
 'Autor',
 'Autor',
 'Autor',
 'Amazon',
 'Autor',
 'Editora',
 'Editora',
 'Autor',
 'Amazon',
 'Amazon',
 'Autor',
 'Autor',
 'E

## 4. Agora, Vamos Verificar a Performance do Treino

# Verificando os verdadeiros positivos

In [101]:
test['Bot'] = lista_teste
tabela_verificação = (pd.crosstab(test.Target , test.Bot , normalize = True )*100).round(3)
tabela_verificação

Bot,Amazon,Autor,Editora
Target,,,
Amazon,14.692,0.474,0.474
Autor,1.422,44.550,2.370
Editora,4.739,21.327,9.953


# Testando o Robô para diferentes situações 

In [102]:
dataframe_final = pd.read_excel('planilha_junta.xlsx')
dataframe_dividido =train_test_split(dataframe_final , test_size =0.4 , shuffle = True)
train_final = dataframe_dividido[0]
test_final = dataframe_dividido[1]
    

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**